In [ ]:
!pip install bitcoin pybitcoin

In [2]:
import binascii
import bitcoin
import pybitcoin
from transactions import transactions

def transaction_hash(transaction):
    return binascii.hexlify(pybitcoin.hash.bin_sha256(pybitcoin.hash.bin_sha256(transaction)))


def ecdsa_raw_sign3(msghash, priv, k):
    hash = bitcoin.hash_to_int(msghash)

    i, j = bitcoin.fast_multiply(bitcoin.G, k)

    x = i
    #   k^-1                      * (H(m) + x * s)
    y = bitcoin.inv(k, bitcoin.N) * (hash + x * bitcoin.decode_privkey(priv)) % bitcoin.N

    return 27 + (j % 2), x, y

In [3]:
alice_priv = "5JKpPhpoVaibqkFdwWzaPoQaHzNTCRmyjyqEpa67G8msKCiyiuA"
alice_address = "1Ppecdv2jWjZjdSJjnQs5JaGhethCsdTCL"
bob_address = "1QFmDXuBr9QY5NoRdbYfZBFFP5cTS9rL4E"

transaction = "Amount:42 From:1Ppecdv2jWjZjdSJjnQs5JaGhethCsdTCL To:1QFmDXuBr9QY5NoRdbYfZBFFP5cTS9rL4E"

timestamp = 1521543600
signature = "HKaLZ/jSgiehh7cyhP5A7AXfEEwuQudjJiJqQLn2qa6Rc9oH1uZ6LztNIFEnG1Lp4EJnNF/RhXgJcky28lD/j6U="

alice = pybitcoin.BitcoinPrivateKey(alice_priv)
print("alice pub", alice.public_key().address())

thash = transaction_hash(transaction)
print(thash)

v, r, s = ecdsa_raw_sign3(thash, alice_priv, timestamp)
sig = bitcoin.encode_sig(v, r, s)
print("sig = %s" % sig)
print("sig = signature: %s" % (sig == signature))

('alice pub', '1Ppecdv2jWjZjdSJjnQs5JaGhethCsdTCL')
18e6d4da1887b0083350f188a29a3895c5755f0e86b84e95ba26eaee0ba9c38a
sig = HKaLZ/jSgiehh7cyhP5A7AXfEEwuQudjJiJqQLn2qa6Rc9oH1uZ6LztNIFEnG1Lp4EJnNF/RhXgJcky28lD/j6U=
sig = signature: True


In [4]:
def find_key_with_collision():
    t = {}
    for [trans, sig] in transactions:
        v, r, s = bitcoin.decode_sig(sig)
        if r in t:
            t[r].append((trans, v,r,s))
        else:
            t[r] = [(trans, v,r,s)]

    collisions = {k:v for k,v in t.iteritems() if len(v) > 1}
    print('%d collision(s)' % len(collisions))

    for k, v in collisions.iteritems():
        if len(v) > 1:
            (t1, v1, r1, s1) = v[0]
            (t2, v2, r2, s2) = v[1]
            z1 = bitcoin.hash_to_int(transaction_hash(t1))
            z2 = bitcoin.hash_to_int(transaction_hash(t2))

            z = z2 - z1
            s = s2 - s1

            r_inv = bitcoin.inv(r1, bitcoin.N)
            s_inv = bitcoin.inv(s, bitcoin.N)

            k = (z * s_inv) % bitcoin.N
            d = (r_inv * (s1 * k - z1)) % bitcoin.N
            return pybitcoin.BitcoinPrivateKey(d)

def sign_transaction():
    alice = find_key_with_collision()
    print(alice.public_key().address())
    tr = 'Amount:1000000 From:1Kx74VzYPdnJ9xxYQRAap4oNsqaAdUdNCA To:14pHuKrkRhLKsW6zxHKQ64DFGyKQffj7GW'
    ts = 1521543600

    v, r, s = ecdsa_raw_sign3(transaction_hash(tr), alice.to_wif(), ts)
    sig = bitcoin.encode_sig(v, r, s)
    print("sig = %s" % sig)

sign_transaction()

1 collision(s)
1Kx74VzYPdnJ9xxYQRAap4oNsqaAdUdNCA
sig = HKaLZ/jSgiehh7cyhP5A7AXfEEwuQudjJiJqQLn2qa6RSzWQeeCrbVIdw0vMpnYHqfUN9CQDI2LKlmHx84jTfjs=
